<a href="https://colab.research.google.com/github/lsmanoel/PowerEletronics/blob/master/driver/driver_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cálculo de Circuito de Comando
Lucas Seara Manoel e Diogo Tavares


![alt text](https://raw.githubusercontent.com/lsmanoel/PowerEletronics/master/driver/images/buck_circuit.PNG)

# Cálculos Preliminares:
$$DutyCycle: D = \frac{V_{out}}{V_{in}}$$
$$I_{out} = \frac{P}{V_{out}}$$ 
$$R_{load} = \frac{V_{out}}{I_{out}}$$
$$\Delta I_L = \frac{V_{out}(1-D)}{fL}$$
$$I_{co} = I_{out}\sqrt{D}$$
$$RSE_{max} = \frac{V_{out}\Delta I_L}{\Delta I_L*I_{co}}$$
$$C_{o} = \frac{I_{out}}{V_{out}*\Delta V_{out}*f}$$

In [0]:
v_in = 36
v_out = 12
f=50e3
L=100e-6
P=100
delta_vout = 0.01

duty_cycle = v_out/v_in
i_out = P/v_out
R_load = v_out/i_out
delta_iL = (v_out*(1-duty_cycle))/(f*i_out*L)

print('f:', f, 'Hz')
print('Duty Cycle:', duty_cycle)
print('i_out:', i_out, 'A')
print('R_load:', R_load, 'ohns')
print('delta_iL:', delta_iL, 'A')

f: 50000.0 Hz
Duty Cycle: 0.3333333333333333
i_out: 8.333333333333334 A
R_load: 1.44 ohns
delta_iL: 0.19199999999999998 A


---
# Transistor

### Esforços na chave

In [0]:
v_key = v_in
print('Tensão na Chave: %.2f V' % v_key)
i_spk = i_out + (delta_vout/2)*i_out
print('Corrente de Pico da Chave: %.2f A' % i_spk)
i_smed = i_out*duty_cycle
print('Corrente Média da Chave: %.2f A' % i_smed)
i_srms = i_out*(duty_cycle ** (1/2))
print('Corrente Eficaz da Chave: %.2f A' % i_srms)

Tensão na Chave: 36.00 V
Corrente de Pico da Chave: 8.38 A
Corrente Média da Chave: 2.78 A
Corrente Eficaz da Chave: 4.81 A


**Transistor Selecionado:**

[SUP40012EL](http://www.vishay.com/docs/76965/sup40012el.pdf)

$V_{ds}$: 40V

Corrente de Pico:  300 A

Corrente de Média:  150 A

# Cálculo de Dissipador

**Perdas por condução:**

$$P_{cond} = R_{DS(ON)}{I_{s_{ef}}}²$$

**Perdas por comutação:**

$$P_{com}=\frac{f}{2}(t_r + t_f)I_{DS(ON)}V_{DS(OFF)}$$

**Perdas Total:**

$$P = P_{cond} + P_{com}$$

**Resistência Térmica:**


*   $R_{ja}$: Junção -> Ambiente
*   $R_{jc}$: Junção -> Encapsulamento
*   $R_{cd}$: Encapsulamento -> Dissipador
*   $R_{da}$: Dissipador -> Ambiente

Resistência térmica máxima para a temperatura da junção e ambiente definidas:

$$R_{ja}=\frac{T_j - T_a}{P}$$

Resistência térmica série do conjunto de elementos entre a junção e o ambiente:

$$R_{ja}=R_{jc}+R_{cd}+R_{da}$$



[Referência](https://electronics.stackexchange.com/questions/16956/to-220-case-to-sink-thermal-resistance) para o valor de resistência térmica entre o encapsulamento e o dissipador:

$R_{cd} = 0.5$ Cº/W

In [0]:
f=50e3
print('f:', f, 'Hz')

Is_ef = 4.81
Rds_on = 0.00179
P_cond = Rds_on*Is_ef**2
print('Perdas por condução:', P_cond, 'W')

t_r = 65e-9
t_f = 18e-9
Ids_on = 8.38
Vds_off=v_in
P_com = (f/2)*(t_r+t_f)*Ids_on*Vds_off
P50 = P_cond+P_com
print('Perdas por comutação:', P_com, 'W')
print('Perdas Totais:', P50, 'W')


f: 50000.0 Hz
Perdas por condução: 0.04141361899999999 W
Perdas por comutação: 0.625986 W
Perdas Totais: 0.667399619 W


In [0]:
T_a = 50
T_j = 125

R_ja = 40 #Datasheet

f=50e3
print('f:', f, 'Hz')

R_ja50 = (T_j - T_a)/P50
print('Resistência Junção->Ambiente Máxima:', R_ja50, 'grausC/W')

print("Precisa de Dissipador?", R_ja50<R_ja)

f: 50000.0 Hz
Resistência Junção->Ambiente Máxima: 112.37645012800044 grausC/W
Resistência Junção->Ambiente Máxima: 112.37645012800044 grausC/W
Precisa de Dissipador? False


**Dissipador escolhido:** 

---

hs3512B com 20 mm

Resistência Térmica: 8,44 ºC / W / 4

Fator de correção para 20 mm: 2.21

$8,44\times2.21=18.65$ Cº/W

[Catálago HS Dissipadores](https://www.hsdissipadores.com.br/catalogo.pdf)

---

EX Series Hearsinks:

Modelo E2A-T220-25E

Resistência Térmica: 16.4ºC / W

[Catálago OHMITE](https://www.mouser.com/datasheet/2/303/sink_ex-1265519.pdf)

---
# Diodo

## Esforços

In [0]:
v_reverse = v_in
print('Tensão Reversa: %.2f V' % v_reverse)

i_dpk = i_out + delta_iL/2*i_out
print('Corrente de Pico do Diodo: %.2f A' % i_dpk)

i_dmed = i_out*(1-duty_cycle)
print('Corrente Média do Diodo: %.2f A' % i_dmed)

i_drms = i_out*((1-duty_cycle) ** (1/2))
print('Corrente Eficaz do Diodo: %.2f A' % i_drms)

Tensão Reversa: 36.00 V
Corrente de Pico do Diodo: 9.13 A
Corrente Média do Diodo: 5.56 A
Corrente Eficaz do Diodo: 6.80 A


**Diodo Selecionado:**

[MBRB3045CT-1G](https://br.mouser.com/ProductDetail/ON-Semiconductor/MBRB3045CT-1G?qs=OSf9jACorvZq6Cx9I44z%252BA%3D%3D)

Peak Reverse Voltage: 45 V
Max Surge Current: 150 A
If - Forward Current: 30 A


---
# Capacitor

In [0]:
i_co = i_out*(duty_cycle ** (1/2))
print('i_co: %.2f A' % i_co)

rse_max = (delta_vout*v_out)/(delta_iL*i_out)
print('rse_max: %.2f Ohm' % rse_max)

C_o = (i_out/(delta_vout*v_out*f)) * duty_cycle
print(f'Capacitância Mínima:{C_o:.2E} F')

i_co: 4.81 A
rse_max: 0.07 Ohm
Capacitância Mínima:4.63E-04 F


**Capacitor Selecionado:**

[667-EEU-FR1H471](https://br.mouser.com/ProductDetail/Panasonic/EEU-FR1H471?qs=sGAEpiMZZMtZ1n0r9vR22f2plZjWOZXTu0LqHMfIqXE%3D)

![alt text](https://raw.githubusercontent.com/lsmanoel/PowerEletronics/master/capacitor/images/capacitor_tab.png)